In [1]:
import json
import os
from tqdm import tqdm

def coco_to_yolo(coco_json_path, output_dir):
    """
    Converts COCO annotations to YOLO format.

    Args:
        coco_json_path (str): Path to the COCO JSON file.
        output_dir (str): Directory where the YOLO-formatted labels and data.yaml will be saved.
    """

    # Load COCO JSON
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Create mapping from category IDs to class names and vice versa
    categories = coco_data['categories']
    category_id_to_name = {cat['id']: cat['name'] for cat in categories}
    category_id_to_index = {cat['id']: idx for idx, cat in enumerate(categories)}  # YOLO class indices start at 0

    # Create mapping from image IDs to file names and image dimensions
    images = coco_data['images']
    image_id_to_info = {img['id']: img for img in images}

    # Prepare output directories
    labels_dir = os.path.join(output_dir, 'labels')
    images_dir = os.path.join(output_dir, 'images')
    os.makedirs(labels_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)

    # Process annotations
    annotations = coco_data['annotations']
    image_to_annotations = {}
    for ann in annotations:
        image_id = ann['image_id']
        if image_id not in image_to_annotations:
            image_to_annotations[image_id] = []
        image_to_annotations[image_id].append(ann)

    # Convert annotations to YOLO format
    for image_id, anns in tqdm(image_to_annotations.items(), desc="Converting annotations"):
        image_info = image_id_to_info[image_id]
        image_width = image_info['width']
        image_height = image_info['height']
        image_file_name = image_info['file_name']
        label_file_name = os.path.splitext(image_file_name)[0] + '.txt'

        # Copy image to images directory
        # Note: Ensure that the images are located where this script can access them.
        # If images are in a different directory, adjust the source path accordingly.
        src_image_path = os.path.join('images', image_file_name)  # Adjust as necessary
        dst_image_path = os.path.join(images_dir, image_file_name)
        if os.path.exists(src_image_path):
            if not os.path.exists(dst_image_path):
                os.link(src_image_path, dst_image_path)
        else:
            print(f"Warning: Image {src_image_path} not found.")

        # Write label file
        label_file_path = os.path.join(labels_dir, label_file_name)
        with open(label_file_path, 'w') as label_file:
            for ann in anns:
                # Get bounding box in COCO format
                bbox_coco = ann['bbox']  # [x_min, y_min, width, height]
                x_min, y_min, width, height = bbox_coco

                # Convert to YOLO format
                x_center = x_min + width / 2
                y_center = y_min + height / 2

                x_center /= image_width
                y_center /= image_height
                width /= image_width
                height /= image_height

                # Get class index (subtract 1 if category IDs start from 1)
                category_id = ann['category_id']
                class_index = category_id_to_index[category_id]

                # Write to label file
                label_file.write(f"{class_index} {x_center} {y_center} {width} {height}\n")

    # Generate data.yaml
    data_yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(data_yaml_path, 'w') as data_yaml_file:
        data_yaml_file.write(f"train: ./train/images\n")
        data_yaml_file.write(f"val: ./valid/images\n")
        data_yaml_file.write(f"test: ./test/images\n\n")
        data_yaml_file.write(f"nc: {len(categories)}\n")
        names = [cat['name'] for cat in categories]
        data_yaml_file.write(f"names: {names}\n")

    print("Conversion completed successfully.")


In [ ]:
# Example usage
coco_json_path = r'd:\OneDrive - Personal\instances_default.json'  # Adjust the path to your COCO JSON file
output_dir = 'yolo'  # Output directory for YOLO-formatted data
coco_to_yolo(coco_json_path, output_dir)

Converting annotations: 100%|██████████| 1496/1496 [00:00<00:00, 1769.78it/s]

Conversion completed successfully.
